<a href="https://colab.research.google.com/github/alanntl/SELGO-LITE/blob/alanntl/SLEGO_LITE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SLEGO Project: UNSW CSE PhD Research - Alan Siu Lung Ng

# Install Libraries

In [1]:
%time
from IPython.display import clear_output
!pip install panel
!pip install openml
!pip install ucimlrepo
!pip install joblib==1.3.2
!pip install openai==1.19.0
!pip install PyPDF2
!pip install pdfkit
!pip install autogluon
!pip install arxiv
!pip install python-docx openpyxl pytesseract Pillow
!pip install sweetviz
!pip install autoviz
!pip install vectorbt
!pip install quantstats
clear_output()

# Import Libraries
Import all the related libraries

In [2]:
import panel as pn
from google.colab import files,drive
import os
import importlib
import sys
import inspect
import ast  # For safely evaluating the input string
import re
import importlib
import json
import io
import time
import param
import json
from datetime import datetime
from google.colab import files
pn.extension(sizing_mode = 'stretch_both')
pn.extension(comms='colab')


# Util Functions

In [ ]:
def delete_func_file(func_file_path):
    # Check if the file exists
    if os.path.exists(func_file_path):
        # Delete the file
        os.remove(func_file_path)
        #print(f"File {func_file_path} has been deleted.")
    else:
        print(f"No file named {func_file_path} found.")

# Load SLEGO Cloud Storage Space (GOOGLE Drive Version)

In [ ]:
# Step 1: Mount Google Drive
drive.mount('/content/drive/',force_remount=True)
# Step 2: Specify the path to the folder containing the .py files in your Google Drive
folder_path = '/content/drive/MyDrive/SLEGO/slegospace'
files.view(folder_path)
# Change the current working directory to the top-level folder
os.chdir(folder_path)
# Now you can reference subfolders and files relative to the top-level folder
dataspace = '/dataspace/'  # This is equivalent to '/content/drive/MyDrive/SLEGO/slegospace/dataspace'
recordspace = '/recordspace/'  # Equivalent to '/content/drive/MyDrive/SLEGO/slegospace/recordspace'
functionspace = '/functionspace/'  # Equivalent to '/content/drive/MyDrive/SLEGO/slegospace/functionspace'
knowledgespace = '/knowledgespace/'  # Equivalent to '/content/drive/MyDrive/SLEGO/slegospace/knowledgespace'

# Select which library to import

In [ ]:
# Step 3: Get the list of .py files in the folder
py_files = [file for file in os.listdir(folder_path+functionspace) if file.endswith('.py')]

# New Step: Check if func.py exists and delete it
func_file_path = 'func.py'
delete_func_file(func_file_path)

funcfilecombo = pn.widgets.MultiChoice(name='Select Function',
                                       value=['util.py',
                                              'func_data_preprocss.py',
                                              'func_moving_avg_plot.py',
                                              'llm.py',
                                              'func_eda.py',
                                              'func_uci_dataset.py',
                                              'webscrape.py',
                                              'func_arxiv.py',
                                              'func_backtest.py',
                                              'func_autogluon.py'],
                                       options=py_files, height=150)

# create funcfilecombo_change function
def funcfilecombo_change(event):
  delete_func_file(func_file_path)

  py_files = funcfilecombo.value
  # Step 4 (Modified): Create a new file named func.py in the current repository
  with open(func_file_path, 'w') as func_file:
      # Step 5: Iterate over each .py file and append its content to func.py
      for py_file in py_files:
          file_path = os.path.join(folder_path+functionspace, py_file)
          with open(file_path, 'r') as file:
              func_file.write(file.read() + '\n')

# param watch
funcfilecombo.param.watch(funcfilecombo_change, 'value')
display(funcfilecombo)

# call the event
funcfilecombo_change(None)


# SLEGO APP

In [ ]:


# Step 6: Import the functions from func.py
import func
importlib.reload(func)

funccombo = func._create_multi_select_combobox(func)
input_text = pn.widgets.TextAreaInput(value='', placeholder='input the parameters', height=500)
compute_btn = pn.widgets.Button(name='Compute', height =50,  button_type='primary')
output_text= pn.widgets.TextAreaInput(value='',placeholder= 'Results will be shown here')
savepipe_btn = pn.widgets.Button(name='Save Pipeline', height =50)
pipeline_text= pn.widgets.TextInput(value='', placeholder='Input Pipeline Name', height=50)

global pipeline_dict
pipeline_dict = {}


def get_doc_string(pipeline):
    text = input_text.value
    output=''
    data = ast.literal_eval(text)
    data.keys()
    # loop keys
    for key in data.keys():
        output +='#######'+str(key)+'#######\n'
        try:
            output += eval(f'func.{key}.__doc__')+'\n'
        except:
            output += 'No docstring found for this function\n'
    return output

def input_text_change(event):
    text = input_text.value
    text = re.sub(r'\bfalse\b', 'False', input_text.value, flags=re.IGNORECASE)
    pipeline_dict = ast.literal_eval(text)

def funccombo_change(event):
    output_text.value = ''
    list_funcs = funccombo.value
    list_params =[]

    for funcchoice in funccombo.value:
      function=  eval('func.'+funcchoice)
      list_params.append(func._extract_parameter(function))

    funcs_params = dict(zip(list_funcs,list_params))
    formatted_data = json.dumps(funcs_params, indent=5)
    input_text.value = str(formatted_data)
    output_text.value = get_doc_string(input_text.value)


def save_record(space, data, pipeline_name=None):
    """
    Saves the given data structure as a JSON file named with the current datetime.

    Parameters:
    - data: The data structure to be saved as JSON.
    """
    if pipeline_name is None:
      # Generate a filename based on the current datetime, e.g., "record_20230405_153000.json"
      filename = datetime.now().strftime("record_%Y%m%d_%H%M%S.json")
    else:
      filename = pipeline_name+'.json'

    # Define the full path where you want to save the file, here assuming current directory
    full_path = './'+space+'/' + filename

    # Serialize and save the data structure to a file
    with open(full_path, "w") as file:
        json.dump(data, file, indent=5)


def compute_btn_clicked(event):

    output_text.value = ''
    # Get the text from the text_input
    text = input_text.value
    text = re.sub(r'\bfalse\b', 'False', input_text.value, flags=re.IGNORECASE)


    # You can now do something with the text, like processing it or displaying results
    # For this example, let's just print it (or display it somewhere in your panel app)
    pipeline_dict = ast.literal_eval(text)


    for function_name, parameters in pipeline_dict.items():
        module = 'func.'
        start_time = time.time()  # Start the timer

        # Dynamically execute the function
        function = eval(module + function_name)
        result = function(**parameters)

        # Computation time
        compute_time = time.time() - start_time

        output_text.value += f"\n===================={function_name}====================\n\n"
        output_text.value += f"Function computation Time: {compute_time:.4f} seconds\n"
        output_text.value += f"\n"
        output_text.value += f"{str(result)}"
    save_record('recordspace', pipeline_dict)

def save_pipeline(event):
  if pipeline_text.value == '':
    pipeline_name = '__'
  else:
    pipeline_name = pipeline_text.value
  text = input_text.value
  text = re.sub(r'\bfalse\b', 'False', input_text.value, flags=re.IGNORECASE)
  data = ast.literal_eval(text)
  save_record('knowledgespace', data, pipeline_name)

# save output into specific folder
funccombo.param.watch(funccombo_change, 'value')
input_text.param.watch(input_text_change, 'value')
compute_btn.on_click(compute_btn_clicked)
savepipe_btn.on_click(save_pipeline)


# App

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='.*findfont.*')
# Creating a Panel column to display the combobox and the text area
# CiO5GzpXbxZQsMuKEQEkT3BlbkFJz4LS3FuI3f5NqmF1BXO
app = pn.Row(pn.Column(funccombo,compute_btn,pn.Row(savepipe_btn,pipeline_text), input_text), pn.Column(output_text))
app